In [94]:
import numpy as np
import librosa as lr
from scipy import signal

In [95]:
# implement a 5.0 HRTFs

HRTF_center, sr = lr.load('CenterChannel.wav'); 
HRTF_left, sr = lr.load('LeftChannel.wav'); 
HRTF_right, sr = lr.load('RightChannel.wav'); 
HRTF_leftsurround, sr = lr.load('LeftSurroundChannel.wav'); 
HRTF_rightsurround, sr = lr.load('RightSurroundChannel.wav'); 

In [96]:
# Create upmixed HRTF with an additional LFE channel
HRTF_upmixed = np.zeros((HRTF_center.shape[0], 6));  # 6 channels for 5.1
HRTF_upmixed[:, :5] = np.column_stack((HRTF_center, HRTF_left, HRTF_right, HRTF_leftsurround, HRTF_rightsurround));
HRTF_upmixed[:, 5] = np.mean(HRTF_upmixed[:, :5], axis=1);  # create an LFE channel

In [97]:
# load surround sound file
surround_soundfile, sr = sf.read('SurroundSound.wav');

In [98]:
# Apply upmixed HRTFs to surround sound channels
center_channel = signal.convolve(surround_soundfile[:, 0], HRTF_upmixed[:, 0], mode='same');
frontleft_channel = signal.convolve(surround_soundfile[:, 1], HRTF_upmixed[:, 1], mode='same');
frontright_channel = signal.convolve(surround_soundfile[:, 4], HRTF_upmixed[:, 2], mode='same');
backleft_channel = signal.convolve(surround_soundfile[:, 2], HRTF_upmixed[:, 3], mode='same');
backright_channel = signal.convolve(surround_soundfile[:, 3], HRTF_upmixed[:, 4], mode='same');
lfe_channel = signal.convolve(surround_soundfile[:, 5], HRTF_upmixed[:, 5], mode='same');

In [99]:
# (1) using HRTFs (room processing optional) create a 2-channel .WAV file with the result

In [100]:
left_stereo = (frontleft_channel + backleft_channel) / np.sqrt(2);
right_stereo = (frontright_channel + backright_channel) / np.sqrt(2);
stereo_mix = np.vstack((left_stereo, right_stereo)).T;

In [101]:
wavfile.write('HZ_2channelwav.wav', sr, stereo_mix);

In [102]:
# (2) create a stereo downmix .WAV file without the VSS processing - downmix the multi-channels to left/right

In [103]:
downmix = np.sum(surround_soundfile, axis=1) / 6;
downmix_stereo = np.tile(downmix, (2, 1)).T;
wavfile.write('HZ_downmix.wav', sr, downmix_stereo);